You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:

I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Brief overview of the latest trends in AI

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare
- AI ethics and regulations

III. Key Players in the AI Industry
- Leading companies in AI technology
- Profiles of key executives in 

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a vital component in numerous industries, transforming the way tasks are executed and decisions are made. With the capability to analyze massive amounts of data and generate predictions based on patterns, AI holds the potential to revolutionize businesses and enhance efficiency. The current trends in AI, ranging from advancements in machine learning to developments in natural language processing, are molding the future of technology and creating new opportunities for innovation.

## Latest Trends in Artificial Intelligence

Machine learning, a subset of AI, is making significant progress in predictive analytics and autonomous systems. Natural language processing, another fundamental aspect of AI, is enhancing communication between humans and machines, resulting in more intuitive interactions. In the healthcare sector, 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become a vital component in numerous industries, transforming the way tasks are executed and decisions are made. With the capability to analyze massive amounts of data and generate predictions based on patterns, AI holds the potential to revolutionize businesses and enhance efficiency. The current trends in AI, ranging from advancements in machine learning to developments in natural language processing, are molding the future of technology and creating new opportunities for innovation.

## Latest Trends in Artificial Intelligence

Machine learning, a subset of AI, is making significant progress in predictive analytics and autonomous systems. Natural language processing, another fundamental aspect of AI, is enhancing communication between humans and machines, resulting in more intuitive interactions. In the healthcare sector, AI is being utilized to analyze medical images, forecast disease outbreaks, and personalize patient care. However, as AI becomes more prevalent, the necessity for ethical guidelines and regulations to govern its utilization is becoming increasingly urgent.

## Key Players in the AI Industry

Prominent companies in the AI technology sector, such as Google, Amazon, and Microsoft, are heavily investing in research and development to maintain a competitive edge. Key executives in the AI sector, such as Sundar Pichai of Google and Satya Nadella of Microsoft, are spearheading innovation and shaping the future of technology. Collaboration and competition among major players are propelling advancements in AI, leading to novel breakthroughs and applications across diverse industries.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research, including the advancement of self-learning algorithms and neural networks, are pushing the boundaries of what is achievable with AI technology. The impact of AI on the job market is a subject of concern, as automation and AI-driven systems are projected to replace certain roles while creating fresh opportunities in other areas. AI applications in addressing global challenges, such as climate change and healthcare disparities, are presenting innovative solutions to intricate problems. Future projections for AI technology encompass the integration of AI into everyday devices and the continuous expansion of the AI industry.

## Target Audience Analysis

Tech enthusiasts intrigued by AI innovations, business professionals seeking to implement AI solutions, students and researchers in the field of artificial intelligence, and the general audience curious about the future of AI can all gain valuable insights by staying abreast of the latest trends and developments in the AI industry. By subscribing to our newsletter, readers can receive regular updates and insights on the rapidly evolving field of artificial intelligence.

In conclusion, artificial intelligence is shaping the future of technology and providing fresh opportunities for innovation across various industries. By keeping up with the latest trends, key players, and noteworthy news in the AI industry, individuals can make informed decisions and stay ahead in this rapidly evolving field. Stay informed and be part of the AI revolution by subscribing to our newsletter today.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan:

Title: Exploring the Latest Trends in Sustainable Fashion

Introduction:
- Brief overview of the growing importance of sustainable fashion in the industry
- Introduce the key players and brands leading the way in sustainable practices

Key Points:
1. Latest Trends in Sustainable Fashion:
- Focus on eco-friendly materials such as organic cotton, Tencel, and recycled fabrics
- Highlight the rise of circular fashion and upcycling in the industry
- Discuss the importance of transparency and ethical practi

I now can edit this blog post to align with the brand's voice and make sure it follows journalistic best practices.

Final Answer:
# Exploring the Latest Trends in Sustainable Fashion

Over the past few years, the fashion industry has experienced a notable shift towards sustainability, as both brands and consumers have become increasingly conscious of the environmental and ethical implications of their clothing choices. This growing emphasis on sustainable fashion is reflected in the rising use of eco-friendly materials like organic cotton, Tencel, and recycled fabrics. Not only do these materials help reduce the carbon footprint of the fashion industry, but they also support a more circular economy by repurposing existing resources.

Circular fashion and upcycling have emerged as significant trends in sustainable fashion, with designers and brands exploring innovative ways to give new life to old garments and minimize waste. Additionally, there is a noticeable focus on transparency an

In [14]:
Markdown(result)

# Exploring the Latest Trends in Sustainable Fashion

Over the past few years, the fashion industry has experienced a notable shift towards sustainability, as both brands and consumers have become increasingly conscious of the environmental and ethical implications of their clothing choices. This growing emphasis on sustainable fashion is reflected in the rising use of eco-friendly materials like organic cotton, Tencel, and recycled fabrics. Not only do these materials help reduce the carbon footprint of the fashion industry, but they also support a more circular economy by repurposing existing resources.

Circular fashion and upcycling have emerged as significant trends in sustainable fashion, with designers and brands exploring innovative ways to give new life to old garments and minimize waste. Additionally, there is a noticeable focus on transparency and ethical practices within the supply chain, driven by consumer demand for greater insight into the production process and the impact of their clothing on the environment and communities.

### Key Players in Sustainable Fashion

Leading the charge in sustainable practices are brands like Patagonia, renowned for their use of recycled materials and advocacy for fair labor practices. Another standout brand is Eileen Fisher, which prioritizes sustainability through initiatives like their take-back program, allowing customers to return old garments for upcycling into new pieces. Stella McCartney is a trailblazer in cruelty-free fashion, utilizing innovative materials such as vegan leather and championing animal welfare in the industry.

Collaborations between designers and sustainable initiatives have become more prevalent, showcasing partnerships that aim to raise awareness of environmental issues and foster sustainable practices in the fashion sector. These collaborations not only drive innovation but also underscore the potential for positive change when creative minds unite for a common purpose.

### Noteworthy News in Sustainable Fashion

Recent advancements in sustainable fashion technology have led to exciting developments, including waterless dyeing techniques and 3D knitting processes that reduce water and energy consumption during production. These innovations not only enhance the sustainability of the fashion industry but also challenge traditional methods, paving the way for a more eco-conscious sector.

The impact of COVID-19 on sustainable fashion practices and consumer awareness has been profound, prompting more individuals to reassess their consumption habits and opt for ethical and sustainable choices. The pandemic has underscored the importance of resilience and adaptability in the fashion industry, compelling brands to prioritize sustainability and social responsibility in their operations.

### Call to Action

As we navigate the ever-evolving landscape of the fashion industry, it is imperative for consumers to support sustainable fashion by choosing ethical brands and educating themselves on sustainable practices. By making informed decisions and advocating for change, we can all contribute to a more sustainable future for the fashion industry and the planet. To delve deeper into sustainable fashion and learn how you can make a meaningful impact, explore resources such as the Sustainable Fashion Institute, Fashion Revolution, Ethical Fashion Forum, and Sustainable Apparel Coalition.

In conclusion, sustainable fashion transcends being a mere trend; it represents a movement towards a more conscious and responsible industry. By staying informed, supporting ethical brands, and embracing innovation, we can all play a role in shaping a sustainable future for fashion and the world. Let's continue to explore, collaborate, and advocate for positive change in the fashion industry.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).